In [25]:
import pandas as pd
import numpy as np


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm, tqdm_notebook, trange
from scipy.optimize import fmin
from pylab import rcParams
from IPython.display import clear_output
import copy
rcParams['figure.figsize'] = 8, 5

pd.set_option('display.max_columns', None)

In [2]:
# Imports for better visualization
from matplotlib import rcParams
#colorbrewer2 Dark2 qualitative color table
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

rcParams['figure.figsize'] = (10, 4)
rcParams['figure.dpi'] = 150
#rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['font.size'] = 8
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'
rcParams['axes.grid'] = True
rcParams['axes.facecolor'] = '#eeeeee'

In [3]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_excel('Sample_Submission.xlsx')
train.shape, test.shape, sample.shape

((947, 54), (527, 53), (527, 1))

In [4]:
cols = ['Freq_Of_Word_4', 'Freq_Of_Word_29', 'Freq_Of_Word_31', 'Freq_Of_Word_32', 'Freq_Of_Word_34', 'Freq_Of_Word_38', 
 'Freq_Of_Word_41']

In [5]:
y = train['IsGoodNews']

del train['IsGoodNews']

data = pd.concat([train, test]).reset_index(drop=True)
data = data.drop(cols, axis=1)
data.shape

(1474, 46)

In [20]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df, deg=2):
    combos = list(combinations(list(df.columns), deg))
    colnames = list(df.columns)+['_'.join(x) for x in combos]
    
    #scaler = MinMaxScaler()
    #scaler.fit(df)
    #df = scaler.transform(df)
    
    poly = PolynomialFeatures(degree = deg, interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indices = [i for i,x in enumerate(list((df==0).all())) if x]
    df= df.drop(df.columns[noint_indices], axis=1)
    
    return df

In [7]:
data_2 = add_interactions(data, degree=2)
data_2.shape

(1474, 1081)

In [8]:
data_2.head()

,Freq_Of_Word_1,Freq_Of_Word_2,Freq_Of_Word_3,Freq_Of_Word_5,Freq_Of_Word_6,Freq_Of_Word_7,Freq_Of_Word_8,Freq_Of_Word_9,Freq_Of_Word_10,Freq_Of_Word_11,...,Freq_Of_Word_49_Freq_Of_Word_50,Freq_Of_Word_49_TotalEmojiCharacters,Freq_Of_Word_49_LengthOFFirstParagraph,Freq_Of_Word_49_StylizedLetters,Freq_Of_Word_50_TotalEmojiCharacters,Freq_Of_Word_50_LengthOFFirstParagraph,Freq_Of_Word_50_StylizedLetters,TotalEmojiCharacters_LengthOFFirstParagraph,TotalEmojiCharacters_StylizedLetters,LengthOFFirstParagraph_StylizedLetters
0,-0.351864,2.620660,1.253645,-0.465210,-0.353977,-0.304257,-0.240708,-0.318797,-0.352968,-0.320591,...,0.012481,-0.002280,-0.003686,0.017641,-0.004525,-0.007314,0.035011,0.001336,-0.006396,-0.010338
1,-0.351864,-0.318036,-0.561952,-0.465210,-0.353977,-0.304257,3.837751,-0.318797,-0.352968,-0.320591,...,0.068929,0.016313,0.029695,0.061983,0.048724,0.088696,0.185138,0.020991,0.043815,0.079759
2,-0.351864,-0.318036,-0.561952,-0.465210,-0.353977,-0.304257,-0.240708,-0.318797,-0.352968,-0.320591,...,0.068929,0.016313,0.028504,0.059637,0.048724,0.085137,0.178129,0.020149,0.042156,0.073661
3,1.210190,2.682528,1.291868,0.221744,-0.353977,-0.304257,0.859101,-0.318797,2.374782,-0.320591,...,-0.051760,-0.202680,-0.344975,-0.091600,0.454593,0.773748,0.205451,3.029835,0.804503,1.369318
4,-0.351864,-0.318036,-0.561952,-0.465210,-0.353977,-0.304257,-0.240708,-0.318797,-0.352968,-0.320591,...,-0.141348,0.004368,0.017186,0.019535,-0.026751,-0.105266,-0.119650,0.003253,0.003697,0.014548


In [11]:
X = data_2[:train.shape[0]]
test_X = data_2[train.shape[0]:].reset_index(drop=True)

X['IsGoodNews'] = y
X.shape

C:\Users\shankar\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(947, 1082)

In [12]:
X.to_csv('train_1081_feat.csv', index=False)
test_X.to_csv('test_1081_feat.csv', index=False)